In [ ]:
#数据读取以及xy提取
import pandas as pd
import matplotlib as plt
#样例数据读取
df = pd.read_excel('')
#根据共线性矩阵,保留与房价相关性最高的日间人口,将夜间人口和夜间20-29岁人口进行比例处理
def age_percent(row):
    if row['夜间人口'] == 0:
        return 0
    else:
        return row['夜间20-29岁人口']/row['夜间人口']
df['per_a20-29'] = df.apply(age_percent,axis=1)
df = df.drop(columns=['夜间人口','夜间20-29岁人口'])
#制作标签变量
price_median = df['房价']median()
df['is_high'] = df['房价'].map(lambda x: True if x >= price_median else False)
print(df['is_high'].value_counts())
#数据集基本情况查看
print(df.shape)
print(df.dtypes)
#划分数据集
x = df[['日间人口','夜间人口','夜间20-29岁人口','','','']]
print(x.shape)
y = df[['is_high']]
print(y.shape)
print(df.isnull().sum())